# deal with cases by modzcta

In [1]:
import pandas

In [2]:
data = pandas.read_csv('../datas/06-cases/data-by-modzcta.csv')
data.shape

(177, 10)

In [3]:
data.head()

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,608,2201.85,27613.09,28,101.40,4.30,12409
1,10002,Chinatown/Lower East Side,Manhattan,1756,2331.30,75322.71,161,213.75,5.85,29657
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,944,1748.87,53977.81,35,64.84,3.04,30400
3,10004,Financial District,Manhattan,70,2355.22,2972.12,1,33.65,3.53,2014
4,10005,Financial District,Manhattan,140,1598.68,8757.23,2,22.84,3.36,3934


In [4]:
data = data[['MODIFIED_ZCTA', 'NEIGHBORHOOD_NAME', 'COVID_CASE_RATE', 'COVID_DEATH_RATE']]
data.shape

(177, 4)

In [5]:
data.head()

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,COVID_CASE_RATE,COVID_DEATH_RATE
0,10001,Chelsea/NoMad/West Chelsea,2201.85,101.40
1,10002,Chinatown/Lower East Side,2331.30,213.75
2,10003,East Village/Gramercy/Greenwich Village,1748.87,64.84
3,10004,Financial District,2355.22,33.65
4,10005,Financial District,1598.68,22.84


In [6]:
data['deathtocase'] = data['COVID_DEATH_RATE'] / data['COVID_CASE_RATE']
data

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,COVID_CASE_RATE,COVID_DEATH_RATE,deathtocase
0,10001,Chelsea/NoMad/West Chelsea,2201.85,101.40,0.046052
1,10002,Chinatown/Lower East Side,2331.30,213.75,0.091687
2,10003,East Village/Gramercy/Greenwich Village,1748.87,64.84,0.037075
3,10004,Financial District,2355.22,33.65,0.014287
4,10005,Financial District,1598.68,22.84,0.014287
...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,5137.88,569.88,0.110917
173,11692,Arverne/Edgemere,3851.80,451.75,0.117283
174,11693,Arverne/Broad Channel,3519.53,228.85,0.065023
175,11694,Belle Harbor-Neponsit/Rockaway Park,4788.40,433.12,0.090452


In [36]:
data.to_csv('../datas/06-cases/data-by-modzcta-merged.csv', index=False)

# add lat and lng

In [8]:
import geopandas

In [9]:
geodata = geopandas.read_file('../datas/00-tractTOzcta/Modified Zip Code Tabulation Areas (MODZCTA).geojson')
geodata.shape

(178, 5)

In [10]:
geodata.head()

,label,modzcta,pop_est,zcta,geometry
0,"10001, 10118",10001,23072,"10001, 10119, 10199","MULTIPOLYGON (((-73.98774 40.74407, -73.98819 ..."
1,10002,10002,74993,10002,"MULTIPOLYGON (((-73.99750 40.71407, -73.99709 ..."
2,10003,10003,54682,10003,"MULTIPOLYGON (((-73.98864 40.72293, -73.98876 ..."
3,10004,10004,3028,10004,"MULTIPOLYGON (((-74.00827 40.70772, -74.00937 ..."
4,10005,10005,8831,"10005, 10271","MULTIPOLYGON (((-74.00783 40.70309, -74.00786 ..."


In [13]:
geodata = geodata.astype({'modzcta': 'int64'})
geodata.dtypes

label         object
modzcta        int64
pop_est       object
zcta          object
geometry    geometry
dtype: object

In [34]:
geodata[geodata['modzcta'] == 10001]['geometry']

0    MULTIPOLYGON (((-73.98774 40.74407, -73.98819 ...
Name: geometry, dtype: geometry

In [25]:
data['lat'] = 0.0
data['lng'] = 0.0
data.loc[0, 'lat']

0.0

In [35]:
for index, row in data.iterrows():
    point = geodata[geodata['modzcta'] == row['MODIFIED_ZCTA']]['geometry'].centroid
    data.loc[index, 'lat'] = float(point.y)
    data.loc[index, 'lng'] = float(point.x)
data

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,COVID_CASE_RATE,COVID_DEATH_RATE,deathtocase,lat,lng
0,10001,Chelsea/NoMad/West Chelsea,2201.85,101.40,0.046052,40.750688,-73.997138
1,10002,Chinatown/Lower East Side,2331.30,213.75,0.091687,40.715780,-73.986174
2,10003,East Village/Gramercy/Greenwich Village,1748.87,64.84,0.037075,40.731826,-73.989159
3,10004,Financial District,2355.22,33.65,0.014287,40.703747,-74.013032
4,10005,Financial District,1598.68,22.84,0.014287,40.706100,-74.008870
...,...,...,...,...,...,...,...
172,11691,Edgemere/Far Rockaway,5137.88,569.88,0.110917,40.601329,-73.761359
173,11692,Arverne/Edgemere,3851.80,451.75,0.117283,40.592797,-73.792196
174,11693,Arverne/Broad Channel,3519.53,228.85,0.065023,40.596457,-73.816711
175,11694,Belle Harbor-Neponsit/Rockaway Park,4788.40,433.12,0.090452,40.577599,-73.844551


In [37]:
data['deathtocase'].min()

0.0

In [38]:
data['deathtocase'].max()

0.17742744940591948